# VERSION 1

In [1]:
#flashcard class
class flashcard():
    def __init__(self, qns, ans, n_correct=0, n_wrong=0, review_time=0):
        self._qns = qns
        self._ans = ans 
        self._n_correct = n_correct
        self._n_wrong = n_wrong
        self._review_time = review_time
        
    def get_qns(self):
        return self._qns
    def get_ans(self):
        return self._ans
    def get_n_correct(self):
        return self._n_correct
    def get_n_wrong(self):
        return self._n_wrong
    def get_review_time(self):
        return self._review_time
    
    def set_qns(self,qns):
        self._qns=qns
    def set_ans(self,ans):
        self._ans=ans
    def set_n_correct(self,n_correct):
        self._n_correct=n_correct
    def set_n_wrong(self,n_wrong):
        self._n_wrong=n_wrong
    def set_review_time(self,review_time):
        self._review_time=review_time

        
    def __str__(self):
        ret= f"{self._qns}, {self._ans}, {self._n_correct}, {self._n_wrong}, {self._review_time}"
        return ret
    
    def __lt__(self, other):
        #Criteria for prioritizing flashcards based on the length of the question
        return len(self._qns) < len(other._qns)
    


In [2]:

####
# This code contains the algorithm to obtain samples from optimal reviewing intensity.
# This script read two csv files, preprocesed_weights.csv, observations_1k.csv and outputs revieiw times.
# For user-item pairs that are not reviewed in the period, we show empty string.
####

import numpy as np

Q = 1.0  # parameter Q defined in eq. 8 in the paper. where Q is a given parameter, which trades off recall probability and number of item reviews—the higher its value, the lower the number of reviews.
T = 10.0  # number of days in the future to generate reviewing timeself.


#this gets the intensity to be used in the sampler function
def intensity(t, n_t, q):
    return 1.0/np.sqrt(q)*(1-np.exp(-n_t*t))

#apparently this function samples from an inhomogeneous Poisson process with intensity and it returns the sampled time
def sampler(n_t, q, T):
    t = 0
    while(True):
        max_int = 1.0/np.sqrt(q)
        t_ = np.random.exponential(1 / max_int)

        if t_ + t > T:
            t=T-1
        t = t+t_
        proposed_int = intensity(t, n_t, q)
        if np.random.uniform(0, 1, 1)[0] < proposed_int / max_int:
            return t

#main function for calculating review time        
def calculate_review_time(flashcard,pqueue):
    
    #n_correct and n_wrong are for each indiv flashcard
    n_correct = flashcard.get_n_correct()
    n_wrong = flashcard.get_n_wrong()
    
    #correct and wrong are for the whole deck average
    correct=float(0)
    wrong=float(0)
    qsize=0
    
    while not pqueue.empty():
        item=pqueue.get()[1]
        qsize+=1
        correct+=int(item.get_n_correct())
        wrong+=int(item.get_n_wrong())
    
    pqueue.put((float(item.get_review_time()),item))
    if qsize!=0:
        correct=correct/qsize
        wrong=wrong/qsize
        

    n_t= 2**(-(correct*n_correct+wrong*n_wrong))
    t_rev = sampler(n_t, Q, T)
    return t_rev

from queue import PriorityQueue
flashcard1=flashcard('What is force?',"Force is the product of mass and acceleration.")
flashcard2=flashcard('hi','bye')
p0= PriorityQueue()
p0.put((2.5,flashcard1))
p0.put((0.3,flashcard2))

print(calculate_review_time(flashcard1,p0))

       


0.3560078585841099


In [3]:
flashcard1=flashcard("konnichiwa","hi")
flashcard2=flashcard('sayonara','bye')
flashcard3=flashcard('ichi','one')
flashcard4=flashcard('ichigo','strawberry')

pnew=PriorityQueue()
pnew.put((2.5,flashcard1))
pnew.put((0.3,flashcard2))
pnew.put((0.2,flashcard3))
pnew.put((4.1,flashcard4))



In [4]:
def show_flashcard(pqueue):
    status=''
    while status != 'stop':
        flashcard_to_show=pqueue.get()[1]
        print(f'{flashcard_to_show.get_qns()}')
        status = input('Type C (correct) or W (wrong), or type "stop" to end: ').lower()
        if status == 'stop':
            break
        elif status == 'c':
            flashcard_to_show.set_n_correct(flashcard_to_show.get_n_correct() + 1)
        elif status == 'w':
            flashcard_to_show.set_n_wrong(flashcard_to_show.get_n_wrong() + 1)
            print('done')
        
        #attain new review time
        review_time = calculate_review_time(flashcard_to_show,pqueue)
        print(review_time)
        
        # Enqueue the modified flashcard back into the priority queue in new position
        pqueue.put((review_time, flashcard_to_show))
        print(pqueue)
    
   
    

# VERSION 2

In [5]:
#flashcard class
class flashcard():
    def __init__(self, qns, ans, ease_record=[], review_time=0):
        self._qns = qns
        self._ans = ans 
        self._ease_record=ease_record
        self._review_time = review_time
        
    def get_qns(self):
        return self._qns
    def get_ans(self):
        return self._ans
    def get_review_time(self):
        return self._review_time
    def get_ease_record(self):
        return self._ease_record
    
    def set_qns(self,qns):
        self._qns=qns
    def set_ans(self,ans):
        self._ans=ans
    def set_review_time(self,review_time):
        self._review_time=review_time
        
    def add_ease_record(self,new_record):
        self._ease_record.append(new_record)

        
    def __str__(self):
        ret= f"{self._qns}, {self._ans}, {self._ease_record}, {self._review_time}"
        return ret
    
    def __lt__(self, other):
        #Criteria for prioritizing flashcards based on the length of the question
        return len(self._qns) < len(other._qns)
    


In [6]:
#TESTING, DELETE LATER
flashcardA=flashcard('hallo','hello')
flashcardA.add_ease_record(2)
flashcardA.add_ease_record(3)
print(flashcardA)

hallo, hello, [2, 3], 0


In [7]:
#this is how to use a priority queue

pnew=PriorityQueue()

flashcard1=flashcard("konnichiwa","hi")
flashcard2=flashcard('sayonara','bye')
flashcard3=flashcard('ichi','one')
flashcard4=flashcard('ichigo','strawberry')

pnew.put((flashcard1.get_review_time(),flashcard1))
pnew.put((flashcard2.get_review_time(),flashcard2))
pnew.put((flashcard3.get_review_time(),flashcard3))
pnew.put((flashcard4.get_review_time(),flashcard4))

print(pnew.queue)


[(0, <__main__.flashcard object at 0x103d43eb0>), (0, <__main__.flashcard object at 0x103d40f40>), (0, <__main__.flashcard object at 0x103d0dcf0>), (0, <__main__.flashcard object at 0x103d0d180>)]


In [8]:
# adapted from https://gist.github.com/doctorpangloss/13ab29abd087dc1927475e560f876797
# Polyglot spaced repetition algorithm
#Based on "SM2+":
#(1 day for incorrect items because you want to test as soon as possible, since clearly the user did not remember well)

"""
Returns the number of days to delay the next review of an item by, fractionally, based on the history of answers x to
a given question, where
x == 0: Incorrect, Very Hard
x == 1: Incorrect, Hard
x == 2: Correct, Easy
x == 3: Correct, Very Easy
@param x The history of answers in the above scoring.
@param theta When larger, the delays for correct answers will increase.
"""
    
def sm2(x: [int], a=6.0, b=-0.8, c=0.28, d=0.02, theta=0.3) -> float:
    assert all(0 <= x_i <= 5 for x_i in x)
    correct_x = [x_i >= 2 for x_i in x]
    # If you got the last question incorrect, just return 1
    if not correct_x[-1]:
        return 1.0
    
    # Calculate the latest consecutive answer streak
    num_consecutively_correct = 0
    for correct in reversed(correct_x):
        if correct:
            num_consecutively_correct += 1
        else:
            break
    
    return a*(max(1.3, 2.5 + sum(b+c*x_i+d*x_i*x_i for x_i in x)))**(theta*num_consecutively_correct)

#testing
x=[1,2,3,2]
sm2(flashcardA.get_ease_record())
 

10.54618121573873

In [ ]:
from IPython.display import clear_output


def show_flashcard(pqueue):
    status=''
    while status!='stop':
        flashcard_to_show=pqueue.get()[1]
        print(flashcard_to_show.get_qns())
        show_answer=input('Input anything to see the answer')
        print('---------------------------------------------------------------------------------------------')
        print(flashcard_to_show.get_ans())
        
        invalid=0
        while invalid!=1:
            new_record = input('Type 0 for Very Hard, 1 for Hard, 2 for Easy and 3 for Very Easy: ')
            if new_record=='0' or new_record=='1' or new_record =='2' or new_record=='3':
                invalid=1
        new_record=int(new_record)
        flashcard_to_show.add_ease_record(new_record)
        new_review_time=sm2(flashcard_to_show.get_ease_record())
        pqueue.put((new_review_time,flashcard_to_show))
        clear_output()
        show_flashcard(pqueue)
        
show_flashcard(pnew)
   
    

ichi
